# Imports

In [3]:
import matplotlib.pyplot as plt
import random
import os
import numpy as np
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import utils, optimizers
from PIL import Image
import shutil
from tensorflow.keras import models, layers
from tensorflow.keras.applications import inception_v3
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import DenseNet169

# Image Data Generator

In [4]:
#These variables can be changes, excluding train_dir

train_dir = "../raw_data/Training"
img_height, img_width = 256, 256
batch_size = 32

In [5]:
#Splits into train_generator and validation_generator
#This bulk uploads the images
#Creates target (y) for us!

#Play around with the interpolation argument - bicubic, lanczos??? 

train_datagen = ImageDataGenerator(rescale=1./255,
                                    vertical_flip=True,
                                    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
                                    train_dir,
                                    target_size=(img_height, img_width),
                                    batch_size=batch_size,
                                    class_mode='categorical',
                                    subset='training',
                                    keep_aspect_ratio=True,
                                    interpolation='lanczos') # set as training data

validation_generator = train_datagen.flow_from_directory(
                                    train_dir, # same directory as training data
                                    target_size=(img_height, img_width),
                                    batch_size=batch_size,
                                    class_mode='categorical',
                                    subset='validation',
                                    keep_aspect_ratio=True,
                                    interpolation='lanczos') # set as validation data

FileNotFoundError: [Errno 2] No such file or directory: '../raw_data/Training'

In [6]:
# Model needs building + transfer learning  

def initialize_model():
    base_model = densenet.DenseNet169(
                        include_top=False,
                        weights='imagenet',
                        input_shape=(img_height, img_width, 3),
                        classifier_activation='softmax')
    
    base_model.trainable = True

    model = models.Sequential([ 
        base_model,
        layers.Flatten(),
        layers.Dropout(0.4),
        layers.Dense(1000,activation="relu"),
        layers.Dropout(0.3),
        layers.Dense(900, activation="relu"),
        layers.Dropout(0.3),
        layers.Dense(800, activation="relu"),
        layers.Dropout(0.3),
        layers.Dense(700, activation="relu"),
        layers.Dense(83, activation="softmax")
    ])
    
    
    opt = optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=opt,
                loss='categorical_crossentropy',
                metrics=['accuracy']) 
    
    return model

In [7]:
model = initialize_model()

94765736/94765736 [==============================] - 19s 0us/step


In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 8, 8, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 131072)            0         
                                                                 
 dense (Dense)               (None, 500)               65536500  
                                                                 
 dense_1 (Dense)             (None, 7)                 3507      
                                                                 
Total params: 89,127,719
Trainable params: 65,540,007
Non-trainable params: 23,587,712
_________________________________________________________________


In [9]:
es = EarlyStopping(patience=1)

In [10]:
#fit model - fit on train_generator (both X and y) and the validation data is validation_generator
history = model.fit(
                train_generator,
                steps_per_epoch = train_generator.samples // batch_size,
                validation_data = validation_generator, 
                validation_steps = validation_generator.samples // batch_size,
                epochs=500,
                callbacks=[es])

NameError: name 'train_generator' is not defined

In [ ]:
def plot_history(history, title='', axs=None, exp_name=""):
    if axs is not None:
        ax1, ax2 = axs
    else:
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    if len(exp_name) > 0 and exp_name[0] != '_':
        exp_name = '_' + exp_name
    ax1.plot(history.history['loss'], label='train' + exp_name)
    ax1.plot(history.history['val_loss'], label='val' + exp_name)
    #ax1.set_ylim(0., 2.2)
    ax1.set_title('loss')
    ax1.legend()

    ax2.plot(history.history['accuracy'], label='train accuracy'  + exp_name)
    ax2.plot(history.history['val_accuracy'], label='val accuracy'  + exp_name)
    #ax2.set_ylim(0.25, 1.)
    ax2.set_title('Accuracy')
    ax2.legend()
    return (ax1, ax2)

In [ ]:
plot_history(history)

In [ ]:
test_dir = "../raw_data/Testing"

test_datagen = ImageDataGenerator(rescale=1./255) # set validation split

test_generator = test_datagen.flow_from_directory(
                                    test_dir,
                                    target_size=(img_height, img_width),
                                    batch_size=batch_size,
                                    class_mode='categorical',
                                    subset='training',
                                    keep_aspect_ratio=True,
                                    interpolation='lanczos') # set as training data

model.evaluate(test_generator)